<a href="https://colab.research.google.com/github/monya-9/deep-learning-practice/blob/main/04_toy_self_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toy Self-Attention 구현

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 1. Scaled Dot-Product Attention (단일 헤드)
class ScaledDotProductAttention(nn.Module):
    """
    Q, K, V를 받아 attention(Q,K,V) = softmax(QK^T / sqrt(d_k)) V 를 계산
    - attn_weights: (B, T_q, T_k)
    - output:       (B, T_q, d_v)  (여기서는 d_v = d_k = head_dim 가정)
    """
    def __init__(self, dropout=0.0):
        super().__init__()
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

    def forward(self, Q, K, V, mask=None):
        # Q,K,V: (B, T, D)
        d_k = Q.size(-1)
        # (B, T, D) @ (B, D, T) -> (B, T, T)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (d_k ** 0.5)

        if mask is not None:
            # mask==0 위치를 매우 작은 값으로 채워 softmax에서 제외
            scores = scores.masked_fill(mask == 0, float("-inf"))

        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        # (B, T, T) @ (B, T, D) -> (B, T, D)
        output = torch.matmul(attn_weights, V)
        return output, attn_weights

- scores: Q와 K의 내적 결과 → 유사도 측정.

- softmax: 토큰 간 가중치 분포로 변환.

- masked_fill: 마스크가 있으면 특정 위치를 무시(-inf).

- @ V: 가중합 결과 반환.

In [3]:
# 2. Multi-Head Self-Attention (간단 구현)
class MultiHeadSelfAttention(nn.Module):
    """
    - d_model: 토큰 임베딩 차원 (예: 128)
    - num_heads: 헤드 수 (예: 4, 8 등)
    - head_dim = d_model // num_heads 이어야 함
    """
    def __init__(self, d_model=128, num_heads=4, dropout=0.0):
        super().__init__()
        assert d_model % num_heads == 0, "d_model은 num_heads로 나누어 떨어져야 합니다."
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # Q/K/V 투영
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)

        # 헤드 결합 후 최종 출력 투영
        self.W_o = nn.Linear(d_model, d_model, bias=False)

        self.attn = ScaledDotProductAttention(dropout=dropout)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

    def _split_heads(self, x):
        # x: (B, T, d_model) -> (B, num_heads, T, head_dim)
        B, T, _ = x.size()
        x = x.view(B, T, self.num_heads, self.head_dim)
        return x.permute(0, 2, 1, 3)

    def _merge_heads(self, x):
        # x: (B, num_heads, T, head_dim) -> (B, T, d_model)
        B, H, T, D = x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(B, T, H * D)

    def forward(self, x, mask=None):
        """
        x: (B, T, d_model)
        mask: (B, 1, 1, T) 또는 (B, 1, T, T) 형태를 지원 (여기서는 생략 가능)
        """
        # 1) 선형 투영
        Q = self.W_q(x)  # (B, T, d_model)
        K = self.W_k(x)
        V = self.W_v(x)

        # 2) 헤드 분할
        Q = self._split_heads(Q)  # (B, H, T, D)
        K = self._split_heads(K)  # (B, H, T, D)
        V = self._split_heads(V)  # (B, H, T, D)

        # 3) 어텐션 계산 (헤드별 처리)
        # ScaledDotProductAttention은 (B, T, D) 입력을 기대하므로,
        # 헤드 차원을 배치로 합쳐서 계산하거나, 각 헤드를 loop로 처리할 수 있음.
        B, H, T, D = Q.size()
        Qf = Q.reshape(B * H, T, D)
        Kf = K.reshape(B * H, T, D)
        Vf = V.reshape(B * H, T, D)

        if mask is not None:
            # mask를 (B, 1, 1, T) -> (B*H, 1, T) 형태로 맞춤
            mask = mask.expand(B, H, mask.size(-2), mask.size(-1)).reshape(B * H, mask.size(-2), mask.size(-1))

        out, attn = self.attn(Qf, Kf, Vf, mask=mask)  # out: (B*H, T, D)
        out = out.view(B, H, T, D)

        # 4) 헤드 결합 + 최종 투영
        out = self._merge_heads(out)              # (B, T, d_model)
        out = self.W_o(out)                       # (B, T, d_model)
        out = self.dropout(out)
        return out, attn.view(B, H, T, T)         # attn: (B, H, T, T)

1. Q/K/V 선형 변환

- self.W_q, self.W_k, self.W_v: 임베딩 공간에서 Q/K/V를 만드는 projection layer.

2. 헤드 분리 (_split_heads)

- (B, T, d_model) → (B, num_heads, T, head_dim)

3. 스케일드 닷프로덕트 어텐션 계산

- 각 head별로 Q/K/V를 넣어 ScaledDotProductAttention 실행.

4. 헤드 결합 (_merge_heads)

- (B, num_heads, T, head_dim) → (B, T, d_model)

5. 출력 투영

- self.W_o: 여러 헤드 결합 결과를 다시 하나의 임베딩 공간으로 투영.

In [4]:
# 3. 간단 사용
B, T, D = 2, 5, 128   # 배치=2, 토큰길이=5, 임베딩=128
x = torch.randn(B, T, D).to(device)

mha = MultiHeadSelfAttention(d_model=D, num_heads=4, dropout=0.1).to(device)
y, attn = mha(x)  # y: (B, T, D), attn: (B, H, T, T)

print("입력 x:", x.shape)
print("출력 y:", y.shape)
print("어텐션 가중치 attn:", attn.shape)  # (배치, 헤드, 토큰, 토큰)

입력 x: torch.Size([2, 5, 128])
출력 y: torch.Size([2, 5, 128])
어텐션 가중치 attn: torch.Size([2, 4, 5, 5])


입력

-  x: (B, T, d_model)
B: 배치 크기, T: 시퀀스 길이, d_model: 임베딩 차원.

출력

- y: (B, T, d_model)
self-attention 적용 후 동일한 차원 유지.

- attn: (B, num_heads, T, T)
각 헤드별 토큰 간 attention score.